In [24]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm

In [25]:
# main page with letters

next_link = "https://marvelcinematicuniverse.fandom.com/wiki/Category:Characters"

characters_link_list = []
while next_link != None:
    response = requests.get(next_link)
    html_page = response.text
    html_tree = BeautifulSoup(html_page)
    results = html_tree.find_all("a", class_="category-page__member-link")
    next_link = html_tree.find(
        "a", class_="category-page__pagination-next wds-button wds-is-secondary")
    next_link = next_link["href"] if next_link != None else None

    for result in results:
        characters_link_list.append(
            [result.text, "https://marvelcinematicuniverse.fandom.com"+result["href"]])


In [26]:
lista_personaggi = pd.DataFrame(characters_link_list, columns=["Nome", "Link"])

In [27]:
lista_personaggi.head()

,Nome,Link
0,A'Lars,https://marvelcinematicuniverse.fandom.com/wik...
1,A. Kennedy,https://marvelcinematicuniverse.fandom.com/wik...
2,A.I.M. President,https://marvelcinematicuniverse.fandom.com/wik...
3,Aaron (Apple Inc.),https://marvelcinematicuniverse.fandom.com/wik...
4,Aaron (Baby),https://marvelcinematicuniverse.fandom.com/wik...


In [28]:
lista_personaggi.shape

(3749, 2)

In [29]:
def cerca_bio(page_tree):
    biography_list = []
    biography_initial = page_tree.find_all(id="Biography")
    description_ended = False
    if(biography_initial):
        next_span = biography_initial[0].parent.find_next()

        while not description_ended:
            if next_span.name != "a" and next_span.name != "sup":
                biography_list.append(next_span.text)

            next_span = next_span.find_next()
            if next_span.name == "h2":
                description_ended = True
            biography_list = list(dict.fromkeys(biography_list))
            lista_personaggi.at[idx, "Biografia"] = biography_list
    return biography_list

In [ ]:
def cerca_serie(page_tree):
    movies_list = []
    tag_lista_film = None
    movies_initial = page_tree.find_all("div", class_="pi-item pi-data pi-item-spacing pi-border-color")
    for element in movies_initial:
        if element["data-source"] == "tv series":
            tag_lista_film = element

    if tag_lista_film != None:
        links = tag_lista_film.find_all("i")
        for link in links:
            movies_list.append(link.text)
    return movies_list

In [34]:
def cerca_film(page_tree):
    movies_list = []
    tag_lista_film = None
    movies_initial = page_tree.find_all("div", class_="pi-item pi-data pi-item-spacing pi-border-color")
    for element in movies_initial:
        if element["data-source"] == "movie":
            tag_lista_film = element
    
    if tag_lista_film != None:
        links = tag_lista_film.find_all("i")
        for link in links:
            movies_list.append(link.text)
    return movies_list

In [37]:
lista_personaggi["Biografia"] = None
lista_personaggi["Film"] = None
lista_personaggi["Serie"] = None

num_iterazioni = 0

for idx, row in tqdm(lista_personaggi.iterrows(), total=lista_personaggi.shape[0]):
    character_page_response = requests.get(row["Link"])
    character_page_tree = BeautifulSoup(character_page_response.text)
    biografia = cerca_bio(character_page_tree)
    film = cerca_film(character_page_tree)
    serie = cerca_serie(character_page_tree)
    lista_personaggi.at[idx, "Biografia"] = biografia
    lista_personaggi.at[idx, "Film"] = film
    lista_personaggi.at[idx, "Serie"] = serie
    num_iterazioni += 1
    if num_iterazioni == 20:
        break

  0%|          | 1/3749 [00:00<31:43,  1.97it/s]

['Avengers: Infinity War']


  0%|          | 2/3749 [00:01<32:12,  1.94it/s]

[]


  0%|          | 3/3749 [00:01<30:28,  2.05it/s]

...

['The Incredible Hulk', 'Captain America: The First Avenger', 'The Avengers', 'Avengers: Age of Ultron', 'Spider-Man: Homecoming']


  1%|          | 19/3749 [00:09<31:35,  1.97it/s]

[]


  1%|          | 19/3749 [00:10<33:31,  1.85it/s]

[]


In [38]:
lista_personaggi.head(20)

,Nome,Link,Biografia,Film
0,A'Lars,https://marvelcinematicuniverse.fandom.com/wik...,"[Biography, Life on Titan, A'Lars fathered two...",[Avengers: Infinity War]
1,A. Kennedy,https://marvelcinematicuniverse.fandom.com/wik...,"[Biography, First Encounter With S.H.I.E.L.D.,...",[]
2,A.I.M. President,https://marvelcinematicuniverse.fandom.com/wik...,"[Biography, Obadiah Stane contacted the A.I.M....",[]
3,Aaron (Apple Inc.),https://marvelcinematicuniverse.fandom.com/wik...,"[Biography, Apple Inc., Assisting Customers, \...",[Captain America: The Winter Soldier]
4,Aaron (Baby),https://marvelcinematicuniverse.fandom.com/wik...,"[Biography, Sophie walked with her son cradled...",[]
5,Aaron James,https://marvelcinematicuniverse.fandom.com/wik...,"[Biography, Court Case, With Matt Murdock as h...",[]
6,Aaron Martinez,https://marvelcinematicuniverse.fandom.com/wik...,"[Biography, Aaron Martinez was a criminal, sel...",[]
7,Abby,https://marvelcinematicuniverse.fandom.com/wik...,"[Biography, Terrigenesis, \n""When the Mist hit...",[]
8,Abby Rose,https://marvelcinematicuniverse.fandom.com/wik...,"[Biography, To be added\n, To be added]",[]
9,Abe Brown,https://marvelcinematicuniverse.fandom.com/wik...,"[Biography, Decathlon Team, \n""Fort Sumter!""""F...",[Spider-Man: Homecoming]
